<a href="https://colab.research.google.com/github/sayashraaj/BTP/blob/main/Gurobi_trial_MDA1_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TO DO**



1. Big M using indicator variable DONE
2. Model is infeasible when t[origin] is used DONE
3. Convert DB to 14 Day DB
4. Build Cost from DB, not dummy
5. Change t[dest] to ? for multiple dests
6. Test on bigger sample network DONE
7. Use variability in sample network

### Pre Processing

In [ ]:
# Importing the DataFrame

import pandas as pd
url = "https://raw.githubusercontent.com/sayashraaj/Dump/main/TrainScheduleDB.csv"
df = pd.read_csv(url, encoding='cp1252')

df.head(60)

,TrainId,TrainName,StationId,StationName,ScheduledArrival,ScheduledDeparture,Distance,Day
0,22981,KOTA - SRIGANGANAGAR SF EXPRESS,KOTA,KOTA JUNCTION,Source,17:20,0.0,1
1,22981,KOTA - SRIGANGANAGAR SF EXPRESS,KPTN,KESHORAI PATAN,17:35,17:37,13.3,1
2,22981,KOTA - SRIGANGANAGAR SF EXPRESS,LKE,LAKHERI,18:07,18:09,60.6,1
3,22981,KOTA - SRIGANGANAGAR SF EXPRESS,IDG,INDARGARH SUMERGANJ MANDI,18:18,18:20,71.4,1
4,22981,KOTA - SRIGANGANAGAR SF EXPRESS,SWM,SAWAI MADHOPUR JUNCTION,19:00,19:15,107.9,1
5,22981,KOTA - SRIGANGANAGAR SF EXPRESS,CKB,CHAUTH KA BARWARA,19:45,19:47,129.9,1
6,22981,KOTA - SRIGANGANAGAR SF EXPRESS,ISA,ISARDA,20:00,20:02,147.8,1
7,22981,KOTA - SRIGANGANAGAR SF EXPRESS,BNLW,BANASTHALI NIWAI,20:21,20:22,173.7,1
8,22981,KOTA - SRIGANGANAGAR SF EXPRESS,DPA,DURGAPURA,21:13,21:16,232.3,1
9,22981,KOTA - SRIGANGANAGAR SF EXPRESS,JP,JAIPUR JUNCTION,21:35,21:45,239.7,1


In [ ]:
stations_list = ["WL", "TPJ", "MAS", "SBC", "NDLS"]
df_filtered = df[df["StationId"].isin(stations_list)]
df_filtered.head(5)

,TrainId,TrainName,StationId,StationName,ScheduledArrival,ScheduledDeparture,Distance,Day
46716,51901,AGRA CANTT. - OLD DELHI PASSENGER,NDLS,NEW DELHI,15:35,15:40,195.0,1
46721,51902,OLD DELHI - AGRA CANT. PASSENGER,NDLS,NEW DELHI,07:15,07:17,3.3,1
49478,1707,JABALPUR - ATARI FARE SPECIAL,NDLS,NEW DELHI,00:20,00:50,910.7,2
49491,1708,ATARI - JABALPUR FARE SPECIAL,NDLS,NEW DELHI,01:30,02:00,473.4,2
49684,2172,JAMMU TAWI - MUMBAI CST AC SF SPECIAL,NDLS,NEW DELHI,16:40,17:00,577.1,1


In [ ]:
# Filtering df to have trains which stop at at least 1 station in the list

TrainId_dict = {}

for row in df_filtered.itertuples():
  TrainId_dict[row.TrainId] = []

TrainId_list = list(TrainId_dict.keys())
print("TrainId_List: ",TrainId_list)

df_filtered_1 = df[df["TrainId"].isin(TrainId_list)]
df_filtered_1.head(50)

NameError: ignored

In [ ]:
# Filtering dictionary to have trains which stop at at least 1 station in the list

for row in df_filtered_1.itertuples():
    if row.StationId in stations_list:
        TrainId_dict[row.TrainId].append(row.StationId)
#         if row.TrainId=="2193":
#             print(row.TrainId, row.ScheduledArrival, row.StationId)
# print(TrainId_dict["2193"])

print(len(TrainId_dict))

TrainId_dict_temp = {}
for TrainId in TrainId_dict:
    if len(TrainId_dict[TrainId])>1:
        TrainId_dict_temp[TrainId] = TrainId_dict[TrainId]
TrainId_dict = TrainId_dict_temp
print(len(TrainId_dict))

980
87


In [ ]:
# Visual sanity check
for TrainId in TrainId_dict:
    print(TrainId, TrainId_dict[TrainId])

2193 ['TPJ', 'WL']
2194 ['WL', 'TPJ']
12007 ['MAS', 'SBC']
12008 ['SBC', 'MAS']
12027 ['MAS', 'SBC']
12028 ['SBC', 'MAS']
12295 ['SBC', 'MAS', 'WL']
12296 ['WL', 'MAS', 'SBC']
12433 ['MAS', 'WL']
12434 ['WL', 'MAS']
12511 ['WL', 'MAS']
12512 ['MAS', 'WL']
12521 ['WL', 'MAS']
12522 ['MAS', 'WL']
12577 ['WL', 'MAS', 'SBC']
12578 ['SBC', 'MAS', 'WL']
12607 ['MAS', 'SBC']
12608 ['SBC', 'MAS']
12609 ['MAS', 'SBC']
12610 ['SBC', 'MAS']
12615 ['MAS', 'WL', 'NDLS']
12616 ['NDLS', 'WL']
12621 ['MAS', 'WL', 'NDLS']
12622 ['NDLS', 'WL', 'MAS']
12625 ['WL', 'NDLS']
12626 ['NDLS', 'WL']
12627 ['SBC', 'NDLS']
12628 ['NDLS', 'SBC']
12639 ['MAS', 'SBC']
12640 ['SBC', 'MAS']
12655 ['WL', 'MAS']
12656 ['MAS', 'WL']
12657 ['MAS', 'SBC']
12658 ['SBC', 'MAS']
12669 ['MAS', 'WL']
12670 ['WL', 'MAS']
12687 ['MAS', 'WL']
12687-Slip ['MAS', 'WL']
12688 ['WL', 'MAS']
12689 ['MAS', 'TPJ']
12690 ['TPJ', 'MAS']
12691 ['MAS', 'SBC']
12692 ['SBC', 'MAS']
12759 ['MAS', 'WL']
12760 ['WL', 'MAS']
12851 ['WL', 'MAS']
12

In [ ]:
# Filtering df again to have trains which stop at at least 2 stations in the list

TrainId_list = list(TrainId_dict.keys())
print("TrainId_List: ",TrainId_list)

df_filtered_1 = df[df["TrainId"].isin(TrainId_list)]
df_filtered_1.head(50)

TrainId_List:  ['2193', '2194', '12007', '12008', '12027', '12028', '12295', '12296', '12433', '12434', '12511', '12512', '12521', '12522', '12577', '12578', '12607', '12608', '12609', '12610', '12615', '12616', '12621', '12622', '12625', '12626', '12627', '12628', '12639', '12640', '12655', '12656', '12657', '12658', '12669', '12670', '12687', '12687-Slip', '12688', '12689', '12690', '12691', '12692', '12759', '12760', '12851', '12852', '12967', '12968', '12969', '12970', '15119', '15120', '16021', '16022', '16031', '16032', '16093', '16094', '16231', '16232', '16317', '16318', '16318-Slip', '16687', '16688', '16688-Slip', '16793', '16794', '22351', '22352', '22403', '22404', '22415', '22416', '22625', '22626', '22631', '22632', '22645', '22646', '22647', '22648', '22681', '22682', '16787', '22688']


,TrainId,TrainName,StationId,StationName,ScheduledArrival,ScheduledDeparture,Distance,Day
49743,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,TEN,TIRUNELVELI JUNCTION,Source,17:45,0.0,1
49744,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,CVP,KOVILPATTI,19:08,19:10,65.2,1
49745,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,SRT,SATUR,19:33,19:35,86.8,1
49746,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,VPT,VIRUDUNAGAR JUNCTION,20:08,20:10,113.7,1
49747,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,MDU,MADURAI JUNCTION,20:55,21:00,157.1,1
49748,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,DG,DINDIGUL JUNCTION,22:00,22:05,219.4,1
49749,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,TPJ,TIRUCHCHIRAPPALLI JUNCTION,00:05,00:10,313.8,2
49750,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,KRR,KARUR JUNCTION,01:30,01:32,390.0,2
49751,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,NMKL,NAMAKKAL,02:13,02:15,423.9,2
49752,2193,TIRUNELVELI - JABALPUR SF FARE SPECIAL,RASP,RASIPURAM,02:40,02:42,449.6,2


In [ ]:
for each in df_filtered_1.itertuples():
  if row.ScheduledDeparture!="Desi"
  timestr = row.ScheduledDeparture
  df_filtered_1.loc[each.Index, 'ScheduledDeparture'] = sum([a*b for a,b in zip([3600,60], map(int,timestr.split(':')))])
# sum([a*b for a,b in zip(ftr, map(int,timestr.split(':')))])
df_filtered_1.head(5)

SyntaxError: ignored

In [ ]:
cdf_filtered_2 = df_filtered_1

df_filtered_2.loc[df_filtered_2["ScheduledArrival"] > df_filtered_2["ScheduledDeparture"], "ScheduledDeparture"] = "hello"
# for row in df_filtered_2.itertuples():
#   if row.ScheduledDeparture=="hello":
#     print("found")
df_filtered_2.head(5)

In [ ]:
# Smaller DataFrame KOTA-SRIGANGANAGAR, 22981
# df1 = df[0:28]
df1 = df[0:56]
df1 = df_filtered_1

In [ ]:
# Declaring Node class

# class Node:
#   def __init__(self, TrainId, TrainName, StationId, StationName, ScheduledArrival, ScheduledDeparture, Day):
#     self.TrainId = TrainId
#     self.TrainName = TrainName
#     self.StationId = StationId
#     self.StationName = StationName
#     self.ScheduledArrival = ScheduledArrival
#     self.ScheduledDeparture = ScheduledDeparture
#     self.Day = Day
#     self.connectedTo = []

class Node:
  def __init__(self, TrainId, TrainName, StationId, StationName, Time, Day):
    self.TrainId = TrainId
    self.TrainName = TrainName
    self.StationId = StationId
    self.StationName = StationName
    self.Time = Time
    self.Day = Day
    self.connectedTo = []

In [ ]:
# Creating and storing nodes in nodesList
nodesList = []

for row in df1.itertuples():
  if row.ScheduledArrival=='Source':
    print("Source detected")
  else:
    nodesList.append(Node(row.TrainId, row.TrainName, row.StationId, row.StationName, row.ScheduledArrival, row.Day))
  if row.ScheduledDeparture=='Destination':
    print("Destination detected")
  else:
    nodesList.append(Node(row.TrainId, row.TrainName, row.StationId, row.StationName, row.ScheduledDeparture, row.Day))

Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destination detected
Source detected
Destin

In [ ]:
# Printing connectedTo []

def printConnectedTo(i):
  print("Node details", nodesList[i].TrainId, nodesList[i].Time, nodesList[i].Day)
  print("Printing connectedTo for Node",i)
  for x in nodesList[i].connectedTo:
    print("Connection:", x.TrainId, x.Time, x.Day)
  print("----------------------")

In [ ]:
# # Linking for same Train

# for i in range(0, len(nodesList)-1):
#   if nodesList[i].TrainId==nodesList[i+1].TrainId and nodesList[i].Time<nodesList[i+1].Time and :
#     nodesList[i].connectedTo.append(nodesList[i+1])
#   printConnectedTo(i)

for i in range(len(nodesList)-1):
  j=i+1
  nodeA = nodesList[i]
  nodeB = nodesList[j]
  # if i==j or nodeA.TrainId!=nodeB.TrainId or nodeA.Time>nodeB.Time or nodeA.Day>nodeB.Day:
  if i==j or nodeA.TrainId!=nodeB.TrainId:
    continue
  else:
    # print("connecting",nodeA.TrainId, nodeA.Time, nodeA.Day,"to", nodeB.TrainId, nodeB.Time, nodeA.Day)
    nodeA.connectedTo.append(nodeB)

In [ ]:
# Connected nodes with same StationId

for i in range(len(nodesList)):
  for j in range(len(nodesList)):
    nodeA = nodesList[i]
    nodeB = nodesList[j]
    if i==j or nodeA.StationId!=nodeB.StationId or nodeA.TrainId==nodeB.TrainId or nodeA.Time>nodeB.Time or nodeA.Day>nodeB.Day:
      continue
    else:
      # print("connecting",nodeA.TrainId, nodeA.Time, nodeA.Day,"to", nodeB.TrainId, nodeB.Time, nodeA.Day)
      nodeA.connectedTo.append(nodeB)

In [ ]:
# for i in range(len(nodesList)):
#   printConnectedTo(i)

In [ ]:
# converting to nodes[]

nodes = []
nodesDict = {}
reverse_nodesDict = {}
for i in range(len(nodesList)):
  nodes.append(i)
  nodesDict[nodesList[i]]=i
  reverse_nodesDict[i]=nodesList[i]

print("Number of nodes: ",len(nodes))
print(nodes[19])

Number of nodes:  5405
19


In [ ]:
# converting to links[(a,b)]
# node 29, 30 have multiple links

links = []

for i in range(len(nodesList)):
  for j in range(len(nodesList[i].connectedTo)):
    links.append((nodesDict[nodesList[i]], nodesDict[nodesList[i].connectedTo[j]]))

print("Number of links: ",len(links))
print(links[1024])

Number of links:  76238
(78, 45)


In [ ]:
# Costs
cost = [[0 for x in nodes] for y in nodes]

for (a,b) in links:
    cost[a][b]=1

### Setup and Installation

In [ ]:
!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp  # import the installed package

# Create an environment with your WLS license
params = {
"WLSACCESSID": 'f5a52c8d-c747-487a-bf9e-fd3f23ec1c72',
"WLSSECRET": '0bff086e-4c28-4979-b1d9-eadc994a435f',
"LICENSEID": 942624,
}
env = gp.Env(params=params)

# Create the model within the Gurobi environment
model = gp.Model(env=env)

from gurobipy import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 942624
Academic license - for non-commercial use only - registered to ce19b113@smail.iitm.ac.in


In [ ]:
def forwardStarMaker(arr):
  ans = {}
  for (a,b) in arr:
    if a in ans:
      ans[a].append(b)
    else:
      ans[a] = [b]

  for n in nodes:
    if n not in ans:
      ans[n]=[]
  return ans

def backwardStarMaker(arr):
  ans = {}
  for (a,b) in arr:
    if b in ans:
      ans[b].append(a)
    else:
      ans[b] = [a]
  for n in nodes:
    if n not in ans:
      ans[n]=[]
  
  # print("**************** backwardStar ****************")
  # for node in nodes:
  #   if node in ans:
  #     print("node",node,":",ans[node])
  #   else:
  #     print("NOT FOUND", node)

  return ans


# Displaying the Solution

# for (a, b) in links: 
#   if x[a,b].x:
#     print('Flow on link ', (a, b), ' is:', x[a, b].x)

def printTvalues(t, x, nodes, links):
  ans = {}
  for (a,b) in links:
    if x[a,b].x:
      print('Flow on link ', (a, b), ' is:', x[a, b].x)
      ans[a]=1
      ans[b]=1
  print("***********Printing T Values**************")
  ans_T_dict = {}
  for n in ans.keys():
    # print("T for",n,":",t[n].x, reverse_nodesDict[n].TrainId, reverse_nodesDict[n].TrainName, reverse_nodesDict[n].Time, reverse_nodesDict[n].Day, reverse_nodesDict[n].StationName)
    ans_T_dict[t[n].x]=n
  for time_index, node_index in sorted(ans_T_dict.items()):
    print(time_index,":",node_index)

In [ ]:
# print(backwardStarMaker(links))
def nodesToVisit_permutation_maker():
  nodesToVisit_permutation = []
  for node in nodesList:
    if node.StationId == 'WL':
      # print(nodesDict[node], node.Time)
      nodesToVisit_permutation.append(nodesDict[node])
  return nodesToVisit_permutation

### Multiple Dest Approach 1

In [ ]:
def gurobi_optimiser_multiple_dest_1(origin, dest, nodesToVisit):

  N = len(nodes)

  m = gp.Model(env=env) # Use this for Gurobi Academic License
  t = m.addVars(nodes, vtype = GRB.INTEGER)
  x = m.addVars(links, vtype = GRB.BINARY)

  forwardStar = forwardStarMaker(links)
  backwardStar = backwardStarMaker(links)

  m.addConstr(sum([x[origin, k] for k in forwardStar[origin]]) == 1, name = "exit from origin")  # exit from origin
  # m.addConstr(sum([x[j, origin] for j in backwardStar[origin]]) == 0, name = "no entry into origin")  # no entry into origin, origin remains first (alternative to using t[origin]==0)
  m.addConstr(sum([x[k, dest] for k in backwardStar[dest]]) == 1, name = "entry to dest") # entry to dest

  m.addConstr(t[origin] == 0.0, name = "t[origin]")

  for (i,j) in links:
    # Original Indicator constraints
    m.addConstr((x[i,j] == 1) >> (t[nodes[j]]-t[nodes[i]]>=cost[i][j]+1), name = "t[i] t[j] LB activity time ["+str(i)+", "+str(j)+"]")
    m.addConstr((x[i,j] == 1) >> (t[nodes[j]]-t[nodes[i]]<=cost[i][j]+1000), name = "t[i] t[j] UB activity time ["+str(i)+", "+str(j)+"]")

  for n in nodes:
      if n not in [origin, dest]:
        m.addConstr(sum([x[n, k] for k in forwardStar[n]]) == sum([x[k, n] for k in backwardStar[n]]), name = "flow conservation ["+str(i)+", "+str(j)+"]")

        if n in nodesToVisit:
          m.addConstr(sum([x[n, k] for k in forwardStar[n]]) == 1)
          # m.addConstr(sum([x[k, n] for k in backwardStar[n]]) >= 1)

  obj = sum([cost[i][j]*x[i,j] for (i,j) in links])
  # obj = t[dest]
  m.Params.outputFlag = 0
  m.setObjective(obj, sense = GRB.MINIMIZE)
  m.optimize()

  if m.status == GRB.OPTIMAL:
      print('model is optimal ✅✅✅ for dest:',dest)
      print('objective value:',obj.getValue())
      # print('objective value:',obj.x)
      print('')
      printTvalues(t, x, nodes, links)
  elif m.status ==GRB.INFEASIBLE:
      print('model is infeasible ❌❌❌')

      # m.computeIIS();
      # for c in m.getConstrs():
      #     if c.IISConstr:
      #         print('%s' % c.constrName)
  elif m.status == GRB.UNBOUNDED:
      print('model is unbounded')
  else:
      print('Not sure')
  return m

In [ ]:
origin = 135
dests = [124, 135, 136, 147, 163, 164, 279, 280, 298, 333, 405, 406, 499, 500, 661, 662, 727, 728, 877, 878, 911, 912, 972, 1019, 1020, 1091, 1092, 1251, 1292, 1597, 1642, 1726, 1727, 1813, 1840, 1841, 1932, 1952, 1953, 2010, 2011, 2088, 2089, 2109, 2182, 2183, 2228, 2229, 2292, 2324, 2325, 2357, 2480, 2494, 2495, 2618, 2619, 2777, 2866, 2867, 3188, 3189, 3370, 4394, 4395, 4414, 4415, 4593, 4634, 4635, 4758, 4812, 4813, 4906, 4907, 5020, 5021, 5116, 5117, 5198, 5199, 5384, 5385]
nodesToVisit_permutation = nodesToVisit_permutation_maker()

# for origin in dests:

for n in nodesToVisit_permutation:
  for dest in dests:
    m = gurobi_optimiser_multiple_dest_1(origin, dest, [n])
    if m.status == GRB.OPTIMAL:
      print("nodetovisit tested: ",n)
    print('-------------------------------------------------------')

model is infeasible ❌❌❌
-------------------------------------------------------
model is infeasible ❌❌❌
-------------------------------------------------------
model is infeasible ❌❌❌
-------------------------------------------------------
model is infeasible ❌❌❌
-------------------------------------------------------
model is infeasible ❌❌❌
-------------------------------------------------------
model is infeasible ❌❌❌
-------------------------------------------------------
model is optimal ✅✅✅ for dest: 279
objective value: 10.0

Flow on link  (39, 1289)  is: 1.0
Flow on link  (135, 1251)  is: 1.0
Flow on link  (1251, 1252)  is: 1.0
Flow on link  (1252, 1345)  is: 1.0
Flow on link  (1289, 1290)  is: 1.0
Flow on link  (1290, 1291)  is: 1.0
Flow on link  (1291, 1292)  is: 1.0
Flow on link  (1292, 279)  is: 1.0
Flow on link  (1345, 1346)  is: 1.0
Flow on link  (1346, 39)  is: 1.0
***********Printing T Values**************
T for 39 : -0.0 2193 TIRUNELVELI - JABALPUR SF FARE SPECIAL  19:4

KeyboardInterrupt: ignored

In [ ]:
origin = 135
dest = 279
n2v = [39,1345]

gurobi_optimiser_multiple_dest_1(origin, dest, n2v)

model is optimal ✅✅✅ for dest: 279
objective value: 10.0

Flow on link  (39, 1289)  is: 1.0
Flow on link  (135, 1251)  is: 1.0
Flow on link  (1251, 1252)  is: 1.0
Flow on link  (1252, 1345)  is: 1.0
Flow on link  (1289, 1290)  is: 1.0
Flow on link  (1290, 1291)  is: 1.0
Flow on link  (1291, 1292)  is: 1.0
Flow on link  (1292, 279)  is: 1.0
Flow on link  (1345, 1346)  is: 1.0
Flow on link  (1346, 39)  is: 1.0
***********Printing T Values**************
0.0 : 135
2.0 : 1251
4.0 : 1252
6.0 : 1345
8.0 : 1346
10.0 : 39
12.0 : 1289
14.0 : 1290
16.0 : 1291
18.0 : 1292
20.0 : 279


<gurobi.Model MIP instance Unnamed: 5408 constrs, 81643 vars, Parameter changes: LicenseID=942624, OutputFlag=0>